<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Basic Transformation
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
Basic Transformation Exercises Solution<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">
<b>
Transformations on data is one of the most common ETL tasks. Transforming data can range in complexity from simply parsing relevant fields to handle null values without affecting downstream operations.<br>
Common transformations include:
    <ul>
        <li>Imputing null or missing data</li>
        <li>Normalizing values</li>
        <li>Duplicate data</li>
        <li>Exploding arrays</li>
        <li>Pivoting DataFrames</li>
        </ul>
    </b>
</span>
<span style="font-family:verdana; font-size:1.0em;"><br>
    <b>Following is included in this notebook</b>:
    <ul>
        <li>Normalizing Data using user defined function and using MinMaxScaler</li>
        <li>Handling Missing Data by dropping rows, replace missing values with placeholder values and using Imputer to replace missing values</li>
        <li>Dropping duplicate records</li>
        <li>Using Pivot to tranpose a column in dataframe</li>
        <li>Exercise 1: Drop duplicate rows from a csv file</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>IF RUNNING IN DATABRICK environment, set the next cell to True</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Basic Transformations").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

<font color='tomato'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Exercise 1: Removing Duplicates<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Removing duplicates that do not completely match could be caused by user "entry/generated" error, schema errors or data corruption.
Perform the following steps:
    <ol>
        <li>Import csv file <b>people-with-dups.csv</b></li>
        <li>Show the dataframe, print schema and count</li>
        <li>Append Columns to filter duplicates, add following columns:</li>
        <ol>
            <li><b>lcFirstName:</b> lower case First Name</li>
            <li><b>lcLastName:</b> lower case Last Name</li>
            <li><b>lcMiddleName:</b> lower case Middle Name</li>
            <li><b>ssnNums:</b> remove '-' from the social security numbers</li>
            <li>Save this to a new DataFrame</li>
        </ol>
        <li>Drop Duplicates:</li>
        <ol>
            <li>Drop duplicates in all columns except (firstName, lastName, middleName, ssn)</li>
            <li>Drop the columns that were added in the step 3</li>
            <li>Save this to a new DataFrame</li>
            <li>Show DataFrame, print the counts</li>
        </ol>
        <li>Compare the counts before and after removing duplicates</li>
    </ol>
</span>

In [ ]:
if DATA_BRICKS == False:
    csvfile = "../datasets/people-with-dups.csv"
else:
    csvfile = "/FileStore/tables/people-with-dups.csv"

In [ ]:
df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("delimiter", ",")
    .csv(csvfile)
)

In [ ]:
# display first 5 rows of dataframe
showDataFrame(df)

In [ ]:
# print schema
df.printSchema()

In [ ]:
# print count
df.count()

In [ ]:
# transformations
from pyspark.sql.functions import col, lower, translate

withColsDF = (df
  .select(col("*"),
    lower(col("firstName")).alias("lcFirstName"),
    lower(col("lastName")).alias("lcLastName"),
    lower(col("middleName")).alias("lcMiddleName"),
    translate(col("ssn"), "-", "").alias("ssnNums")
))

In [ ]:
showDataFrame(withColsDF)

In [ ]:
# drop duplicates
dedupedDF = (withColsDF
  .dropDuplicates(["lcFirstName", "lcMiddleName", "lcLastName", "ssnNums", "gender", "birthDate", "salary"])
  .drop("lcFirstName", "lcMiddleName", "lcLastName", "ssnNums")
)

In [ ]:
showDataFrame(dedupedDF)

In [ ]:
dedupedDF.count()